In [16]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os

import warnings
warnings.filterwarnings('ignore')

### Data preprocessing

In [17]:

image_size = (224, 224)
batch_size = 32

In [18]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [19]:
train_generator = datagen.flow_from_directory(
    "data/",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="training"
)

Found 160 images belonging to 2 classes.


In [20]:
val_generator = datagen.flow_from_directory(
    "data/",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"
)

Found 40 images belonging to 2 classes.


### Hypermodel function

In [21]:
def build_cnn_model(hp):
    
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=image_size + (3,))
    base_model.trainable = False  # Transfer learning

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())

    # Hidden Dense Layer
    units = hp.Int("units", min_value=64, max_value=256, step=32)
    model.add(Dense(units=units,
                    activation=hp.Choice("activation", ['relu', 'tanh']),
                    kernel_initializer=hp.Choice("initializer", ['glorot_uniform', 'he_normal']),
                    kernel_regularizer=regularizers.l2(hp.Float("lambda", 1e-4, 1e-2, sampling='log'))))

    model.add(Dropout(hp.Float("dropout", 0.2, 0.5, step=0.1)))

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    # Optimizer
    learning_rate = hp.Float("learning_rate", 1e-4, 1e-2, sampling='log')
    optimizer_choice = hp.Choice("optimizer", ['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate)

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

### Tuner setup

In [22]:
tuner = kt.RandomSearch(
    build_cnn_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='cnn_tuning',
    project_name='VGG16_binary'
)

### Early stopping

In [23]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)

### Tuner search

In [24]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop],
    verbose=2
)

Trial 10 Complete [00h 08m 18s]
val_loss: 0.5302904844284058

Best val_loss So Far: 0.5302904844284058
Total elapsed time: 02h 15m 27s


### Get best model and hyperparameters

In [25]:
best_model = tuner.get_best_models(1)[0]
best_hyperparams = tuner.get_best_hyperparameters(1)[0]

In [26]:
print("\nBest Hyperparameters:")
print(best_hyperparams.values)


Best Hyperparameters:
{'units': 224, 'activation': 'tanh', 'initializer': 'glorot_uniform', 'lambda': 0.0001999433153242281, 'dropout': 0.2, 'learning_rate': 0.007522684510012844, 'optimizer': 'rmsprop'}


In [27]:
# Save best model
os.makedirs("model", exist_ok=True)
best_model.save("model/VGG16_best_tuned_model.h5")

### Evaluate Accuracy on Validation/Test Data

In [28]:
# Evaluate model on validation data
val_loss, val_accuracy = best_model.evaluate(val_generator, verbose=1)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.8313 - loss: 0.5215
Validation Accuracy: 82.50%
Validation Loss: 0.5253
